# Strava Activity Data Analysis with Pandas

This notebook demonstrates how to perform database queries using pandas instead of raw SQL. We'll replicate the functionality from the `get_activities()` method in the StravaDatabase class using pandas operations.

## 1. Import Required Libraries

First, let's import all the necessary libraries for data manipulation and database connectivity.

In [2]:
import pandas as pd
import sqlite3
import json
from datetime import datetime
import numpy as np

# For better display of DataFrames
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

Libraries imported successfully!
Pandas version: 2.3.2
NumPy version: 2.3.3


## 2. Load Database Connection and Setup

Establish connection to the SQLite database and set up pandas to work with the database.

In [3]:
# Database path (adjust if needed)
db_path = "strava_data.db"

# Create connection to SQLite database
try:
    conn = sqlite3.connect(db_path)
    print(f"Successfully connected to database: {db_path}")
    
    # Check if tables exist
    tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
    tables = pd.read_sql_query(tables_query, conn)
    print("\nAvailable tables:")
    print(tables)
    
except sqlite3.Error as e:
    print(f"Error connecting to database: {e}")
    
# Check database file size and basic info
import os
if os.path.exists(db_path):
    size_mb = os.path.getsize(db_path) / (1024 * 1024)
    print(f"\nDatabase file size: {size_mb:.2f} MB")

Successfully connected to database: strava_data.db

Available tables:
         name
0    athletes
1  activities

Database file size: 0.59 MB


## 3. Read Activities Data into DataFrame

Use pandas.read_sql_query() to load the entire activities table into a DataFrame.

In [4]:
# Load all activities data into a DataFrame
# This is equivalent to: SELECT * FROM activities
activities_df = pd.read_sql_query("""
    SELECT *
    FROM activities
""", conn)

print(f"Loaded {len(activities_df)} activities from database")
print(f"DataFrame shape: {activities_df.shape}")
print(f"Memory usage: {activities_df.memory_usage(deep=True).sum() / 1024:.2f} KB")

# Display basic info about the DataFrame
print("\nDataFrame Info:")
print(activities_df.info())

# Show first few rows
print("\nFirst 5 rows:")
activities_df.head()

Loaded 217 activities from database
DataFrame shape: (217, 13)
Memory usage: 482.29 KB

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   activity_id           217 non-null    int64  
 1   athlete_id            217 non-null    object 
 2   name                  217 non-null    object 
 3   type                  217 non-null    object 
 4   start_date            217 non-null    object 
 5   distance              217 non-null    float64
 6   moving_time           217 non-null    int64  
 7   elapsed_time          217 non-null    int64  
 8   total_elevation_gain  217 non-null    float64
 9   average_speed         217 non-null    float64
 10  max_speed             217 non-null    float64
 11  raw_data              217 non-null    object 
 12  created_at            217 non-null    object 
dtypes: float64(4), int64(

,activity_id,athlete_id,name,type,start_date,distance,moving_time,elapsed_time,total_elevation_gain,average_speed,max_speed,raw_data,created_at
0,13240840560,117211639,Start with the right stroke,Swim,2025-01-01T17:37:07Z,1000.0,1175,1725,0.0,0.851,5.0,"{""resource_state"": 2, ""athlete"": {""id"": 117211...",2025-09-28 10:05:25
1,13248703520,117211639,Evening Weight Training,WeightTraining,2025-01-02T17:43:30Z,0.0,3106,3106,0.0,0.000,0.0,"{""resource_state"": 2, ""athlete"": {""id"": 117211...",2025-09-28 10:05:25
2,13257300481,117211639,Evening Weight Training,WeightTraining,2025-01-03T17:45:49Z,0.0,2172,2172,0.0,0.000,0.0,"{""resource_state"": 2, ""athlete"": {""id"": 117211...",2025-09-28 10:05:25
3,13276798215,117211639,Evening Weight Training,WeightTraining,2025-01-05T17:11:30Z,0.0,3201,3201,0.0,0.000,0.0,"{""resource_state"": 2, ""athlete"": {""id"": 117211...",2025-09-28 10:05:25
4,13292644686,117211639,Evening Weight Training,WeightTraining,2025-01-06T17:15:04Z,0.0,2601,2601,0.0,0.000,0.0,"{""resource_state"": 2, ""athlete"": {""id"": 117211...",2025-09-28 10:05:25


In [13]:
activities_df.loc[206, 'raw_data']

'{"resource_state": 2, "athlete": {"id": 117211639, "resource_state": 1}, "name": "PRC - slow and steady", "distance": 18152.1, "moving_time": 6483, "elapsed_time": 6483, "total_elevation_gain": 117.0, "type": "Run", "sport_type": "Run", "workout_type": 0, "id": 15714275257, "start_date": "2025-09-06T07:14:51Z", "start_date_local": "2025-09-06T09:14:51Z", "timezone": "(GMT+01:00) Europe/Prague", "utc_offset": 7200.0, "location_city": null, "location_state": null, "location_country": null, "achievement_count": 0, "kudos_count": 31, "comment_count": 0, "athlete_count": 16, "photo_count": 0, "map": {"id": "a15714275257", "summary_polyline": "k}wpHomgwA{@qBg@eAkCwDqCeCgDyAiCe@a@AWFk@\\\\qBUaCd@}CPm@HUIa@c@q@}DsEZo@JmAIQUGU@SNa@@m@UoBKgCGW[K[`@G^[n@s@fAw@`AqCfEkBdCsClEs@|@c@x@yApBiC|D[n@iAdBi@dA_@dAG\\\\Yl@S|@[tBQj@SxF@tDErBDdDBpKAtAOlEIpEw@zHKbEGx@Mt@c@xAKn@@N^zA@r@Ez@\\\\rAJ`JJnB@hADX@x@NhA@x@t@pHAROj@@d@LbA^f@DRhAfNLtBVpB`A~KFhBDtEEjFSzFA`BItBeA~LUhBa@~DSlAOf@W|AEF[hCs@rDo@tB_@bAWxAo@lGW

In [15]:
# Parse JSON and extract desired fields
df_extracted = activities_df["raw_data"].apply(json.loads).apply(
    lambda x: {
        "start_latlng": x["start_latlng"],
        "end_latlng": x["end_latlng"],
        "start_date_local": x["start_date_local"],
        "athlete_count": x["athlete_count"],
        "photo_count": x["photo_count"],
        "kudos_count": x["kudos_count"],
    }
)

In [19]:
df_final = pd.DataFrame(df_extracted.tolist())

In [20]:
df_final

,start_latlng,end_latlng,start_date_local,athlete_count,photo_count,kudos_count
0,[],[],2025-01-01T18:37:07Z,1,0,6
1,[],[],2025-01-02T18:43:30Z,1,0,3
2,[],[],2025-01-03T18:45:49Z,1,0,2
3,[],[],2025-01-05T18:11:30Z,1,0,1
4,[],[],2025-01-06T18:15:04Z,1,0,1
...,...,...,...,...,...,...
212,"[50.097416, 14.462274]","[50.096583, 14.461106]",2025-09-20T09:08:41Z,14,0,21
213,"[50.071274, 14.414223]","[50.066477, 14.415304]",2025-09-21T15:06:17Z,1,0,36
214,[],[],2025-09-26T20:09:12Z,1,0,5
215,[],[],2025-09-22T19:02:53Z,1,0,1


## 4. Filter Activities by Athlete ID

Use pandas filtering operations to select activities for a specific athlete_id instead of using WHERE clause in SQL.

In [4]:
# First, let's see what athlete IDs we have
print("Available athlete IDs:")
athlete_counts = activities_df['athlete_id'].value_counts()
print(athlete_counts)

# Choose an athlete_id for analysis (use the first one if data exists)
if not activities_df.empty:
    # Get the athlete with the most activities for demo
    target_athlete_id = athlete_counts.index[0]
    print(f"\nAnalyzing data for athlete_id: {target_athlete_id}")
    
    # This is equivalent to: WHERE athlete_id = target_athlete_id
    athlete_activities = activities_df[activities_df['athlete_id'] == target_athlete_id].copy()
    
    print(f"Found {len(athlete_activities)} activities for this athlete")
    print(f"Percentage of total activities: {len(athlete_activities)/len(activities_df)*100:.1f}%")
    
else:
    print("No activities found in database")
    target_athlete_id = None
    athlete_activities = pd.DataFrame()

Available athlete IDs:
athlete_id
117211639    217
Name: count, dtype: int64

Analyzing data for athlete_id: 117211639
Found 217 activities for this athlete
Percentage of total activities: 100.0%


## 5. Sort and Limit Results

Apply pandas sort_values() and head()/tail() methods to order by start_date and limit results.

In [5]:
if not athlete_activities.empty:
    # Sort by start_date in descending order (most recent first)
    # This is equivalent to: ORDER BY start_date DESC
    sorted_activities = athlete_activities.sort_values('start_date', ascending=False)
    
    print("Activities sorted by date (most recent first):")
    print(f"Date range: {sorted_activities['start_date'].min()} to {sorted_activities['start_date'].max()}")
    
    # Limit results (equivalent to LIMIT in SQL)
    limit = 10
    limited_activities = sorted_activities.head(limit)
    
    print(f"\nShowing top {limit} most recent activities:")
    print(limited_activities[['activity_id', 'name', 'type', 'start_date', 'distance']].to_string())
    
    # Compare with pandas nlargest (alternative way to get top N)
    print(f"\nAlternative: Using nlargest() to get top {limit} by date:")
    # Convert start_date to datetime for proper sorting
    athlete_activities['start_date_dt'] = pd.to_datetime(athlete_activities['start_date'])
    top_by_date = athlete_activities.nlargest(limit, 'start_date_dt')
    print(top_by_date[['activity_id', 'name', 'type', 'start_date', 'distance']].to_string())
    
else:
    print("No activities to sort and limit")

Activities sorted by date (most recent first):
Date range: 2025-01-01T17:37:07Z to 2025-09-27T07:36:28Z

Showing top 10 most recent activities:
     activity_id                                 name            type            start_date  distance
216  15950715770                         PRC - autumn             Run  2025-09-27T07:36:28Z   16090.6
214  15950715337              Evening Weight Training  WeightTraining  2025-09-26T18:09:12Z       0.0
215  15950715377              Evening Weight Training  WeightTraining  2025-09-22T17:02:53Z       0.0
213  15890630473  Glad that season is going to finish            Ride  2025-09-21T13:06:17Z   65994.9
212  15873587382                 PRC - kind of summer             Run  2025-09-20T07:08:41Z   15015.4
211  15867460571              Evening Weight Training  WeightTraining  2025-09-19T16:34:28Z       0.0
210  15817269821                           Lunch Swim            Swim  2025-09-15T09:37:54Z     895.6
209  15765970540              Evening We

## 6. Data Type Conversions and Cleaning

Convert data types using pandas methods like to_datetime(), astype(), and handle missing values.

In [6]:
if not athlete_activities.empty:
    # Create a working copy for data cleaning
    clean_activities = athlete_activities.copy()
    
    print("Original data types:")
    print(clean_activities.dtypes)
    print("\n" + "="*50)
    
    # Convert start_date to datetime
    clean_activities['start_date'] = pd.to_datetime(clean_activities['start_date'])
    print("✅ Converted start_date to datetime")
    
    # Convert numeric columns to proper types
    numeric_cols = ['distance', 'moving_time', 'elapsed_time', 'total_elevation_gain', 
                   'average_speed', 'max_speed']
    
    for col in numeric_cols:
        if col in clean_activities.columns:
            clean_activities[col] = pd.to_numeric(clean_activities[col], errors='coerce')
            print(f"✅ Converted {col} to numeric")
    
    # Convert distance from meters to kilometers
    clean_activities['distance_km'] = clean_activities['distance'] / 1000
    
    # Convert time from seconds to hours
    clean_activities['moving_time_hours'] = clean_activities['moving_time'] / 3600
    clean_activities['elapsed_time_hours'] = clean_activities['elapsed_time'] / 3600
    
    # Calculate pace (min/km) from average_speed (m/s)
    clean_activities['pace_min_per_km'] = np.where(
        clean_activities['average_speed'] > 0,
        (1000 / clean_activities['average_speed']) / 60,
        np.nan
    )
    
    print("\n✅ Added derived columns: distance_km, moving_time_hours, elapsed_time_hours, pace_min_per_km")
    
    # Handle missing values
    print(f"\nMissing values per column:")
    missing_values = clean_activities.isnull().sum()
    print(missing_values[missing_values > 0])
    
    print("\nUpdated data types:")
    print(clean_activities.dtypes)
    
else:
    print("No activities to clean")

Original data types:
activity_id                           int64
athlete_id                           object
name                                 object
type                                 object
start_date                           object
distance                            float64
moving_time                           int64
elapsed_time                          int64
total_elevation_gain                float64
average_speed                       float64
max_speed                           float64
raw_data                             object
created_at                           object
start_date_dt           datetime64[ns, UTC]
dtype: object

✅ Converted start_date to datetime
✅ Converted distance to numeric
✅ Converted moving_time to numeric
✅ Converted elapsed_time to numeric
✅ Converted total_elevation_gain to numeric
✅ Converted average_speed to numeric
✅ Converted max_speed to numeric

✅ Added derived columns: distance_km, moving_time_hours, elapsed_time_hours, pace_min_per_km

M

## 7. Parse JSON Raw Data

Use pandas apply() with json.loads() to parse the raw_data column and extract additional fields.

In [7]:
if not athlete_activities.empty and 'raw_data' in athlete_activities.columns:
    # Function to safely parse JSON
    def safe_json_parse(json_str):
        try:
            if pd.isna(json_str) or json_str == '':
                return {}
            return json.loads(json_str)
        except (json.JSONDecodeError, TypeError):
            return {}
    
    # Parse raw_data JSON column
    print("Parsing raw_data JSON column...")
    clean_activities['parsed_raw_data'] = clean_activities['raw_data'].apply(safe_json_parse)
    
    # Extract specific fields from raw data
    def extract_field(row, field_name, default=None):
        raw_data = row['parsed_raw_data']
        return raw_data.get(field_name, default)
    
    # Extract additional fields that might be in raw_data
    fields_to_extract = [
        'achievement_count', 'athlete_count', 'average_heartrate', 'max_heartrate',
        'average_cadence', 'has_kudos', 'kudos_count', 'comment_count',
        'pr_count', 'total_photo_count', 'workout_type'
    ]
    
    extracted_fields = {}
    for field in fields_to_extract:
        clean_activities[f'raw_{field}'] = clean_activities.apply(
            lambda row: extract_field(row, field), axis=1
        )
        
        # Count non-null values
        non_null_count = clean_activities[f'raw_{field}'].notna().sum()
        extracted_fields[field] = non_null_count
    
    print("\\nExtracted fields from raw_data:")
    for field, count in extracted_fields.items():
        print(f"  {field}: {count} non-null values")
    
    # Show sample of parsed data
    if len(clean_activities) > 0:
        print("\\nSample of first activity's raw data keys:")
        first_raw_data = clean_activities.iloc[0]['parsed_raw_data']
        if first_raw_data:
            print(f"  Available keys: {list(first_raw_data.keys())[:10]}...")  # Show first 10 keys
    
else:
    print("No raw_data column found or no activities to parse")

Parsing raw_data JSON column...
\nExtracted fields from raw_data:
  achievement_count: 217 non-null values
  athlete_count: 217 non-null values
  average_heartrate: 212 non-null values
  max_heartrate: 212 non-null values
  average_cadence: 62 non-null values
  has_kudos: 0 non-null values
  kudos_count: 217 non-null values
  comment_count: 217 non-null values
  pr_count: 217 non-null values
  total_photo_count: 217 non-null values
  workout_type: 178 non-null values
\nSample of first activity's raw data keys:
  Available keys: ['resource_state', 'athlete', 'name', 'distance', 'moving_time', 'elapsed_time', 'total_elevation_gain', 'type', 'sport_type', 'id']...


## 8. Display Results and Basic Statistics

Show the final DataFrame and compute basic statistics using pandas describe(), info(), and value_counts() methods.

In [8]:
if not athlete_activities.empty:
    print("📊 FINAL RESULTS AND STATISTICS")
    print("=" * 50)
    
    # Select key columns for final display
    key_columns = ['activity_id', 'name', 'type', 'start_date', 'distance_km', 
                   'moving_time_hours', 'average_speed', 'pace_min_per_km', 
                   'total_elevation_gain']
    
    # Filter columns that exist
    display_columns = [col for col in key_columns if col in clean_activities.columns]
    final_df = clean_activities[display_columns].sort_values('start_date', ascending=False)
    
    print(f"\\n🏃‍♂️ Activities for Athlete {target_athlete_id}")
    print(f"Total activities: {len(final_df)}")
    print(f"Date range: {final_df['start_date'].min().date()} to {final_df['start_date'].max().date()}")
    
    # Show top 10 most recent activities
    print(f"\\n📋 Top 10 Most Recent Activities:")
    display_df = final_df.head(10).round(2)
    print(display_df.to_string(index=False))
    
    print(f"\\n📈 SUMMARY STATISTICS")
    print("-" * 30)
    
    # Numerical statistics
    numeric_stats = final_df.select_dtypes(include=[np.number]).describe().round(2)
    print("\\nNumerical columns statistics:")
    print(numeric_stats)
    
    # Activity type distribution
    print(f"\\n🏃 Activity Types Distribution:")
    type_counts = clean_activities['type'].value_counts()
    print(type_counts)
    
    # Monthly activity summary
    if 'start_date' in clean_activities.columns:
        clean_activities['year_month'] = clean_activities['start_date'].dt.to_period('M')
        monthly_stats = clean_activities.groupby('year_month').agg({
            'activity_id': 'count',
            'distance_km': 'sum',
            'moving_time_hours': 'sum'
        }).round(2)
        monthly_stats.columns = ['Activities', 'Total_Distance_km', 'Total_Time_hours']
        
        print(f"\\n📅 Monthly Activity Summary:")
        print(monthly_stats.tail(6))  # Show last 6 months
    
    # Performance metrics
    print(f"\\n🎯 KEY PERFORMANCE METRICS:")
    print(f"  • Total Distance: {clean_activities['distance_km'].sum():.1f} km")
    print(f"  • Total Moving Time: {clean_activities['moving_time_hours'].sum():.1f} hours")
    print(f"  • Average Distance per Activity: {clean_activities['distance_km'].mean():.1f} km")
    print(f"  • Average Pace: {clean_activities['pace_min_per_km'].mean():.1f} min/km")
    print(f"  • Total Elevation Gain: {clean_activities['total_elevation_gain'].sum():.0f} m")
    
else:
    print("❌ No activities found for analysis")

# Close database connection
conn.close()
print(f"\\n✅ Database connection closed")

📊 FINAL RESULTS AND STATISTICS
\n🏃‍♂️ Activities for Athlete 117211639
Total activities: 217
Date range: 2025-01-01 to 2025-09-27
\n📋 Top 10 Most Recent Activities:
 activity_id                                name           type                start_date  distance_km  moving_time_hours  average_speed  pace_min_per_km  total_elevation_gain
 15950715770                        PRC - autumn            Run 2025-09-27 07:36:28+00:00        16.09               1.51           2.95             5.64                 166.0
 15950715337             Evening Weight Training WeightTraining 2025-09-26 18:09:12+00:00         0.00               0.93           0.00              NaN                   0.0
 15950715377             Evening Weight Training WeightTraining 2025-09-22 17:02:53+00:00         0.00               0.72           0.00              NaN                   0.0
 15890630473 Glad that season is going to finish           Ride 2025-09-21 13:06:17+00:00        65.99               2.50          

/var/folders/gk/pv0rf4cx2nn2mmg36_x6cqfh0000gp/T/ipykernel_17358/1726054712.py:38: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  clean_activities['year_month'] = clean_activities['start_date'].dt.to_period('M')


## 🎯 Summary: SQL vs Pandas Comparison

Here's how the pandas operations map to the original SQL query:

### Original SQL (from `get_activities` method):
```sql
SELECT activity_id, name, type, start_date, distance, 
       moving_time, elapsed_time, total_elevation_gain,
       average_speed, max_speed, raw_data
FROM activities 
WHERE athlete_id = {athlete_id}
ORDER BY start_date DESC
LIMIT {limit}
```

### Pandas Equivalent:
```python
# Load data
df = pd.read_sql_query("SELECT * FROM activities", conn)

# Filter by athlete_id (WHERE clause)
athlete_df = df[df['athlete_id'] == target_athlete_id]

# Sort by start_date descending (ORDER BY)
sorted_df = athlete_df.sort_values('start_date', ascending=False)

# Limit results (LIMIT clause)
final_df = sorted_df.head(limit)

# Select specific columns (SELECT clause)
result = final_df[['activity_id', 'name', 'type', 'start_date', 
                   'distance', 'moving_time', 'elapsed_time', 
                   'total_elevation_gain', 'average_speed', 'max_speed', 'raw_data']]
```

### Advantages of Pandas Approach:
- 🔄 **More flexible filtering**: Can apply complex conditions easily
- 📊 **Built-in statistics**: `.describe()`, `.value_counts()`, etc.
- 🧹 **Data cleaning**: Built-in handling of missing values, type conversion
- 📈 **Visualization ready**: Direct integration with matplotlib, seaborn, plotly
- 🔗 **Method chaining**: Clean, readable code with method chaining
- 🚀 **Performance**: Vectorized operations for large datasets